In [9]:
from aif360.metrics.classification_metric import ClassificationMetric
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import warnings

from utils import make_dataset, log_fairness_metrics

warnings.filterwarnings('ignore')

TARGET_COLS = 'two_year_recid'
COMPAS_SCORES_COLS = 'decile_score'
NUMERICAL_FEATURE_COLS = ['age',
                          'juv_fel_count','juv_misd_count','juv_other_count',
                          'priors_count','jail_time']
CATEGORICAL_FEATURE_COLS = ['sex','race',
                            'c_charge_degree']
FEATURE_NAMES = NUMERICAL_FEATURE_COLS+CATEGORICAL_FEATURE_COLS

PROTECTED_COLS = ['sex','race']

BIAS_INFO = {'favorable_label':0,
             'unfavorable_label':1,
             'protected_columns':['race'],
            }

PRIVILEGED_INFO = {'unprivileged_groups':[{'race': 2},
                                          {'race': 1},
                                          {'race': 4},
                                          {'race': 5},
                                          {'race': 6}],
                   'privileged_groups':[{'race': 3}]
                  }

data = pd.read_csv('../data/processed/compas-scores-two-years-processed.csv')
data.head()

,age,juv_fel_count,juv_misd_count,juv_other_count,priors_count,jail_time,sex,race,c_charge_degree,decile_score,two_year_recid,compas_score
0,69,0,0,0,0,0.0,1,1,1,1,0,0.1
1,34,0,0,0,0,10.0,1,2,1,3,1,0.3
2,24,0,0,1,4,1.0,1,2,1,4,1,0.4
3,23,0,1,0,1,0.0,1,2,1,8,0,0.8
4,43,0,0,0,2,0.0,1,1,1,1,0,0.1


In [10]:
train, test = train_test_split(data, test_size=0.2, random_state=1234)

X_train, y_train = train[FEATURE_NAMES], train[TARGET_COLS]
X_test, y_test = test[FEATURE_NAMES], test[TARGET_COLS]

In [11]:
clf = LogisticRegression(random_state=1234)
clf.fit(X_train, y_train)

y_train_pred = clf.predict_proba(X_train)
train['recid_score'] = (y_train_pred[:,1])
train['recid_prediction'] = (y_train_pred[:,1] >0.5).astype(int)

y_test_pred = clf.predict_proba(X_test)
test['recid_score'] = (y_test_pred[:,1])
test['recid_prediction'] = (y_test_pred[:,1] >0.5).astype(int)

In [12]:
cols = FEATURE_NAMES+[TARGET_COLS, 'recid_prediction','recid_score']

ground_truth_train = make_dataset(train[cols], 'two_year_recid', **BIAS_INFO, **PRIVILEGED_INFO)
prediction_train = make_dataset(train[cols], 'recid_prediction', **BIAS_INFO, **PRIVILEGED_INFO)

ground_truth_test = make_dataset(test[cols], 'two_year_recid', **BIAS_INFO, **PRIVILEGED_INFO)
prediction_test = make_dataset(test[cols], 'recid_prediction', **BIAS_INFO, **PRIVILEGED_INFO)

# Equal Odds

In [5]:
from aif360.algorithms.postprocessing import EqOddsPostprocessing

calibrator = EqOddsPostprocessing(**PRIVILEGED_INFO)

calibrator.fit(ground_truth_train, prediction_train)
prediction_test = calibrator.predict(prediction_test)

roc_auc = roc_auc_score(y_test, prediction_test.labels)
clf_metric = ClassificationMetric(ground_truth_test, prediction_test,**PRIVILEGED_INFO)

In [6]:
import neptune
from neptunecontrib.api.utils import get_filepaths

neptune.init('jakub-czakon/model-fairness')
neptune.create_experiment(name='baseline',tags=['postprocessing','race','equal-odds'], 
                          upload_source_files=get_filepaths(extensions=['.py','.ipynb']))

neptune.log_metric('roc_auc',roc_auc)
log_fairness_metrics(clf_metric)

neptune.stop()

https://ui.neptune.ml/jakub-czakon/model-fairness/e/MOD-81


Invalid metric value: inf for channel false_positive_rate_ratio. Metrics with nan or +/-inf values will not be sent to server
Invalid metric value: inf for channel false_discovery_rate_ratio. Metrics with nan or +/-inf values will not be sent to server


# Calibrated Equal Odds

In [6]:
from aif360.algorithms.postprocessing import CalibratedEqOddsPostprocessing

calibrator = CalibratedEqOddsPostprocessing(**PRIVILEGED_INFO)

calibrator.fit(ground_truth_train, prediction_train)
prediction_test = calibrator.predict(prediction_test)

roc_auc = roc_auc_score(y_test, prediction_test.labels)
clf_metric = ClassificationMetric(ground_truth_test, prediction_test,**PRIVILEGED_INFO)

In [7]:
import neptune
from neptunecontrib.api.utils import get_filepaths

neptune.init('jakub-czakon/model-fairness')
neptune.create_experiment(name='baseline',tags=['postprocessing','race','calibrated-equal-odds'], 
                          upload_source_files=get_filepaths(extensions=['.py','.ipynb']))

neptune.log_metric('roc_auc',roc_auc)
log_fairness_metrics(clf_metric)

neptune.stop()

https://ui.neptune.ml/jakub-czakon/model-fairness/e/MOD-82


# Rejection Option

How to pass prediction to dataset

In [13]:
from aif360.algorithms.postprocessing import RejectOptionClassification

calibrator = RejectOptionClassification(**PRIVILEGED_INFO)

calibrator.fit(ground_truth_train, prediction_train)
prediction_test = calibrator.predict(prediction_test)

roc_auc = roc_auc_score(y_test, prediction_test.labels)
clf_metric = ClassificationMetric(ground_truth_test, prediction_test,**PRIVILEGED_INFO)

In [14]:
import neptune
from neptunecontrib.api.utils import get_filepaths

neptune.init('jakub-czakon/model-fairness')
neptune.create_experiment(name='baseline',tags=['postprocessing','race','rejection-option'], 
                          upload_source_files=get_filepaths(extensions=['.py','.ipynb']))

neptune.log_metric('roc_auc',roc_auc)
log_fairness_metrics(clf_metric)

neptune.stop()

https://ui.neptune.ml/jakub-czakon/model-fairness/e/MOD-83
